In [ ]:
import random
import networkx as nx


class Individual:
    def __init__(self, sets, graph):

        self.size = sum(len(s) for s in sets)
        self.graph = graph
        self.sorted_edges = list(self.graph.edges())
        # self.sorted_edges = sorted(self.sorted_edges, key=lambda x:(x[0],x[1]))
        self.sorted_edges = [tuple(sorted(t)) for t in self.sorted_edges]
        self.sorted_edges = sorted(self.sorted_edges, key=lambda x: (x[0], x[1]))
        self.gene = {i: (None, None) for i in self.sorted_edges}
        self.sets = [{tuple(sorted(t)) for t in s} for s in sets]

        self._populate_individual(self.sets)

    def _populate_individual(self, sets):
        visited = {t: 0 for t in self.sorted_edges}

        for set in sets:
            random_set = list(set)
            random.shuffle(random_set)
            for index,cur_edge in enumerate(set):
                # if cur_edge[0]>cur_edge[1]:
                #     set[index] = (cur_edge[1], cur_edge[0])
                #     cur_edge = set[index]
                value_assigned = False
                for other_edge in random_set:
                    if visited[other_edge] or other_edge == cur_edge:
                        continue
                    if self.adjacent(cur_edge, other_edge):
                        self.gene[cur_edge] = other_edge
                        visited[other_edge] = 1
                        value_assigned=True
                        break
                if not value_assigned:
                    for other_edge in random_set:
                        if not visited[other_edge]:
                            self.gene[cur_edge] = other_edge
                            visited[other_edge] = 1
                            break

    def adjacent(self, t1, t2):
        if t1[0] == t2[0] or t1[0] == t2[1] or t1[1] == t2[0] or t1[1] == t2[1]:
            return True
        return False

    def decode(self):
        labels = {l: 0 for l in self.sorted_edges}
        c = 1
        for locus, gene_value in self.gene.items():
            gene_tuple=tuple(gene_value)
            if labels[locus] == 0 and labels[gene_tuple] == 0:
                labels[locus] = c
                labels[gene_tuple] = c
                c += 1
            elif labels[locus] == 0:
                labels[locus] = labels[gene_tuple]
            elif labels[gene_tuple] == 0:
                labels[gene_tuple] = labels[locus]

        value_to_keys = {}
        for key, value in labels.items():
            if value in value_to_keys:
                value_to_keys[value].add(key)
            else:
                value_to_keys[value] = {key}
        return list(value_to_keys.values())

    def get_gene(self, index):
        return self.gene[self.sorted_edges[index]]

    def set_gene(self, index, gene_value):
        self.gene[self.sorted_edges[index]] = gene_value

    def get_loci(self,index):
        return self.sorted_edges[index]

    def __eq__(self, other):

        for key in self.gene:
            if self.gene[key] != other.gene[key] : return False
        return True            

        # for i in range(len(self.gene)):
        #     if (self.gene[i] != other.gene[i] or self.sorted_edges[i] != other.loci[i]):
        #         return False
        # return True

    def __str__(self):
        loci_str = ', '.join(map(str, self.gene.keys()))
        gene_str = ', '.join(map(str, self.gene.values()))

        return f"Loci: [{loci_str}]\nGene: [{gene_str}]"


